In [1]:
# May be necessary for Linux based OS
# !sudo apt-get install ffmpeg

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from process_vad_data import *

/home/fjia/anaconda3/envs/vad/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/fjia/anaconda3/envs/vad/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [4]:
import os
import sys
import glob
import json

import pickle
import wrapt
import inspect
import librosa

In [7]:
data_dir = '/home/fjia/data/freesound_resampled/'

## Process google speech command dataset

In [8]:
sc_data_folder = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2/'

They have provided splited datalist. We follow that.

 [TODO] _background_noise_

In [95]:
process_google_speech_train(sc_data_folder)

Overall: 105835, Train: 84849, Validatoin: 9981, Test: 11005


In [10]:
data_dir = sc_data_folder
out_dir = './manifest'

skip_num_val, seg_num_val = load_list_write_manifest(data_dir, out_dir, 'validation_list.txt', 'sc')
skip_num_test, seg_num_test = load_list_write_manifest(data_dir, out_dir, 'testing_list.txt', 'sc')
skip_num_train, seg_num_train = load_list_write_manifest(data_dir, out_dir, 'training_list.txt', 'sc')

print(f'Val: Skip {skip_num_val} samples. Get {seg_num_val} segments!')
print(f'Test: Skip {skip_num_test} samples. Get {seg_num_test} segments!')
print(f'Train: Skip {skip_num_train} samples. Get {seg_num_train} segments!')

=== Finished preparing manifest ! Skip 0 samples ===
=== Writing 9981 to ./manifest/sc_validation_manifest.json ===
=== Finished preparing manifest ! Skip 0 samples ===
=== Writing 11005 to ./manifest/sc_testing_manifest.json ===
=== Finished preparing manifest ! Skip 0 samples ===
=== Writing 84843 to ./manifest/sc_training_manifest.json ===
Val: Skip 0 samples. Get 9981 segments!
Test: Skip 0 samples. Get 11005 segments!
Train: Skip 0 samples. Get 84843 segments!


In [18]:
data_dir = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2_noisy/'
out_dir = '/home/fjia/data/manifest'
skip_num_train, seg_num_train = load_list_write_manifest(data_dir, out_dir, 'testing_list.txt', 'noisy_sc')

print(f'Train: Skip {skip_num_train} samples. Get {seg_num_train} segments!')

=== Finished preparing manifest ! Skip 0 samples ===
=== Writing 11005 to /home/fjia/data/manifest/noisy_sc_testing_manifest.json ===
Train: Skip 0 samples. Get 11005 segments!


## Process freesound dataset

In [32]:
split_dataset_class(data_dir)

File /home/fjia/data/freesound_resampled/all.txt exists. Overwrite it!
File /home/fjia/data/freesound_resampled/background.txt exists. Overwrite it!
File /home/fjia/data/freesound_resampled/speech.txt exists. Overwrite it!
=== 5837 total samples! 1849 speech samples, 3988 background samples! ===
Finished split and write to file by class !


split_train_val_test(data_dir, 'all.txt' 0.20 ,  0.10)

In [33]:
split_train_val_test(data_dir, 'speech.txt',  0.10 ,  0.10)
split_train_val_test(data_dir, 'background.txt',  0.10 ,  0.10)

Overall: 1849, Train: 1479, Validatoin: 185, Test: 185
Finished split train, val and test for speech.txt. Write to files !
Overall: 3988, Train: 3190, Validatoin: 399, Test: 399
Finished split train, val and test for background.txt. Write to files !


duration = 60.0 would lead to skipping too many samples (2k/5k)

## freesound
### speech inside freesound / NOT USE NOW

### background inside freesound / WE WANT IT

In [8]:
data_dir = '/home/fjia/data/freesound_resampled/'
out_dir = './manifest'

skip_num_val, seg_num_val = load_list_write_manifest(data_dir, out_dir, 'background_validation_list.txt', '1s', 1, 1)
skip_num_test, seg_num_test = load_list_write_manifest(data_dir, out_dir, 'background_testing_list.txt', '1s', 1, 1)
skip_num_train, seg_num_train = load_list_write_manifest(data_dir, out_dir, 'background_training_list.txt', '1s', 1, 1)

print(f'Val: Skip {skip_num_val} samples. Get {seg_num_val} segments!')
print(f'Test: Skip {skip_num_test} samples. Get {seg_num_test} segments!')
print(f'Train: Skip {skip_num_train} samples. Get {seg_num_train} segments!')

outdir ./manifest does not exist. Creat directory.
=== Finished preparing manifest ! Skip 142 samples ===
=== Writing 7222 to ./manifest/1s_background_validation_manifest.json ===
=== Finished preparing manifest ! Skip 146 samples ===
=== Writing 6885 to ./manifest/1s_background_testing_manifest.json ===
=== Finished preparing manifest ! Skip 1129 samples ===
=== Writing 57368 to ./manifest/1s_background_training_manifest.json ===
Val: Skip 142 samples. Get 7222 segments!
Test: Skip 146 samples. Get 6885 segments!
Train: Skip 1129 samples. Get 57368 segments!


files = sorted(glob.glob(data_dir + '/*/*.wav'))
write_manifest(data_dir, files, manifest_name='noise_manifest_100', duration_max=1.0, filter_long=True, duration_limit=100.0)

## construct  manifest for train, val and test

## Background + Speech Command 57k, 7k, 7k

In [14]:
sc_dir = './manifest/'
get_clean_max_json(sc_dir, out_dir, '100ms_sc_training_manifest.json', 570000, '2balanced')
get_clean_max_json(sc_dir, out_dir, '100ms_sc_validation_manifest.json', 70000, '2balanced')
get_clean_max_json(sc_dir, out_dir, '100ms_sc_testing_manifest.json', 70000, '2balanced')

Get 570000 speech command to ./manifest/2balanced_100ms_sc_training_manifest.json from speech commands
Get 70000 speech command to ./manifest/2balanced_100ms_sc_validation_manifest.json from speech commands
Get 70000 speech command to ./manifest/2balanced_100ms_sc_testing_manifest.json from speech commands


In [23]:
sc_dir = './manifest/'
get_clean_max_json(sc_dir, out_dir, '100ms_background_training_manifest.json', 570000, '2balanced')
get_clean_max_json(sc_dir, out_dir, '100ms_background_validation_manifest.json', 70000, '2balanced')
get_clean_max_json(sc_dir, out_dir, '100ms_background_testing_manifest.json', 70000, '2balanced')

Get 565503/570000 speech command to ./manifest/2balanced_100ms_background_training_manifest.json from speech commands
Get 70001/70000 speech command to ./manifest/2balanced_100ms_background_validation_manifest.json from speech commands
Get 67828/70000 speech command to ./manifest/2balanced_100ms_background_testing_manifest.json from speech commands


In [19]:
sc_dir = '/home/fjia/data/manifest/'

get_clean_max_json(sc_dir, out_dir, 'noisy_sc_testing_manifest.json', 7000, '2balanced')

Get 7001/7000 speech command to /home/fjia/data/manifest/2balanced_noisy_sc_testing_manifest.json from speech commands


files = sorted(glob.glob(data_dir + '/*/*.wav'))
write_manifest('.', files, manifest_name='noise_cls_manifest', duration_max=1.0, filter_long=True, duration_limit=10.0)

sc_dir = '/home/fjia/NEMO/examples/asr/notebooks/manifest'
get_clean_max_json(sc_dir, 'background_training_manifest.json', 17000)
get_clean_max_json(sc_dir, 'background_validation_manifest.json', 2000)
get_clean_max_json(sc_dir, 'background_testing_manifest.json', 5000)

## Combine test json

combined_des = './manifest/all_test.json'
manifest_to_combine  = './manifest/background_testing_manifest.json,./manifest/2balanced_sc_testing_manifest.json'

combine_test_set(manifest_to_combine, combined_des)

In [20]:
combined_des = '/home/fjia/data/old_manifest/noisy_all_test.json'
manifest_to_combine  = '/home/fjia/data/manifest/2balanced_noisy_sc_testing_manifest.json,/home/fjia/data/old_manifest/background_testing_manifest.json,/home/fjia/data/old_manifest/2balanced_sc_testing_manifest.json'

combine_test_set(manifest_to_combine, combined_des)

/home/fjia/data/manifest/2balanced_noisy_sc_testing_manifest.json
/home/fjia/data/old_manifest/background_testing_manifest.json
/home/fjia/data/old_manifest/2balanced_sc_testing_manifest.json


20887

In [60]:
clean_speech_file = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2/right/22aa3665_nohash_0.wav'
noise_file = '/home/fjia/data/freesound_resampled/Car/id_152941 RainyDayDriveToandWalkInTheWood.wav'
clean_data_dir = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2/'
out_dir = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2_noisy/'

output_mixed_file = make_noisy_speech(clean_speech_file, noise_file, 
                      clean_data_dir, noise_dir, out_dir, 1, 20)


## “multistyle training” (MTR) data augmentation technique 

In [7]:
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
clean_speech_file_list_path = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2/training_list.txt'
noise_file_list_path = '/home/fjia/data/freesound_resampled/background_training_list.txt'

In [8]:
sc_data_dir = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2/'

In [22]:
import os

clean_data_dir = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2/'
noise_dir = '/home/fjia/data/freesound_resampled/'
out_dir = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2_noisy/'
all_snr = []

if not os.path.exists(out_dir):
    print('Create output dir for noisy output!')
    os.mkdir(out_dir)

def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

subdir = get_immediate_subdirectories(clean_data_dir)

for i in subdir:
    if not os.path.exists(os.path.join(out_dir, i)):
        os.mkdir(os.path.join(out_dir, i))

Create output dir for noisy output!


In [28]:
import time
start = time.time()

import os

from process_vad_data import * 
clean_data_dir = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2/'
noise_dir = '/home/fjia/data/freesound_resampled/'
out_dir = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2_noisy/'
all_snr = []

if not os.path.exists(out_dir):
    print('Create output dir for noisy output!')
    os.mkdir(out_dir)

def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

subdir = get_immediate_subdirectories(clean_data_dir)

for i in subdir:
    if not os.path.exists(os.path.join(out_dir, i)):
        os.mkdir(os.path.join(out_dir, i))
        
        
from multiprocessing import Pool
    
with open(clean_speech_file_list_path) as f:
    clean_speech_file_list = f.read().splitlines() 

clean_speech_file_list  = clean_speech_file_list[:5]##


with open(noise_file_list_path) as f:
    noise_file_list = f.read().splitlines() 
    

def process_one_file(clean_speech_file):
    noise_file = random.choice(noise_file_list)
     ## [TODO]Adjust SNR 0-30 from google paper
    output_mixed_file, snr = make_noisy_speech(clean_speech_file, noise_file, 
                                               clean_data_dir, noise_dir, out_dir, 0, 30, None) 
    output_mixed_file_shortpath =  output_mixed_file.split(out_dir)[1]
    
#     with open('/home/fjia/data/google_dataset_v2/google_speech_recognition_v2_noisy/training_list.txt', 'a') as f:
#         f.write(output_mixed_file_shortpath) 
#         f.write('\n')
        
    return output_mixed_file_shortpath


p = Pool(processes=18)
output_mixed_file_shortpath = p.map(process_one_file, clean_speech_file_list)
p.close()

np.save('output.npz', output_mixed_file_shortpath)
end = time.time()

In [30]:
print(output_mixed_file_shortpath)

In [15]:
import time
start = time.time()
clean_data_dir = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2/'
noise_dir = '/home/fjia/data/freesound_resampled/'
out_dir = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2_noisy/'
all_snr = []
with open(clean_speech_file_list_path) as f:
    clean_speech_file_list = f.read().splitlines() 

    
with open(noise_file_list_path) as f:
    noise_file_list = f.read().splitlines() 
    
for clean_speech_file in clean_speech_file_list:
    #random select a noise file
    
    noise_file = random.choice(noise_file_list)
    ## [TODO]Adjust SNR 0-30 from google paper
    output_mixed_file, snr = make_noisy_speech(clean_speech_file, noise_file, 
                                               clean_data_dir, noise_dir, out_dir, 0, 30, None) 
    output_mixed_file_shortpath =  output_mixed_file.split(out_dir)[1]
    all_snr.append(snr)
    print(snr)
    if len(all_snr)%10 == 0:
        print(f'{len(all_snr)}===========')
    with open('/home/fjia/data/google_dataset_v2/google_speech_recognition_v2_noisy/training_list.txt', 'a') as f:
        f.write(output_mixed_file_shortpath) 
        f.write('\n')
# print(output_mixed_file)

np.save('snr.npz', all_snr)
end = time.time()

Create output dir for noisy output!
Create output subdir for noisy output!
Duplicate noise!
Duplicate noise!
10
8
24
Duplicate noise!
28
12
13
16
Duplicate noise!
30
9
29
10===========
28
15
5
22
1
12
3
24
2
0
20===========
28
8
29
Duplicate noise!
6
28
24
27
8
Duplicate noise!
17
30
30===========
Duplicate noise!
Duplicate noise!
Duplicate noise!
30
22
26
24
9
21
30
18
18
18
40===========
4
2
22
23
12
22
26
27
3
5
50===========
4
Duplicate noise!
Duplicate noise!
3
14
4
30
26
20
10
14
3
60===========
30
4
4
27
10
23
29
2
11
Duplicate noise!
Duplicate noise!
30
70===========
30
16
2
19
16
Duplicate noise!
14
6
13
20
27
80===========
26
30
29
19
27
7
17
14
29
5
90===========
16
3
Duplicate noise!
Duplicate noise!
22
17
17
7
21
27
12
28
100===========
21
21
18
26
27
2
19
0
14
20
110===========
10
3
29
8
19
25
16
Duplicate noise!
Duplicate noise!
15
12
Duplicate noise!
25
120===========
26
30
16
21
Duplicate noise!
6
Duplicate noise!
Duplicate noise!
18
3
18
28
5
130===========
13
22
19
1

KeyboardInterrupt: 

In [86]:

y1, sample_rate = librosa.load(output_mixed_file, mono=True)

# REPRODUCE
ipd.Audio(y1, rate = sample_rate)

In [46]:
np.load('output.npy')

In [47]:
a['a']

In [53]:
np.save('/tmp/123', np.array([[1, 2, 3], [4, 5, 6]]), allow_Pickle=True)
np.load('/tmp/123.npy')

In [ ]:
'/home/fjia/data/haorizi.wav'

In [9]:
sample_rate

16000

In [12]:
noise_dir = '/home/fjia/data/freesound_resampled/'
out_dir = '.'
all_snr = []


    
with open(noise_file_list_path) as f:
    noise_file_list = f.read().splitlines() 

with open(noise_file_list_path) as f:
    noise_file_list = f.read().splitlines() 
    

def process_one_file(clean_speech_file):
    noise_file = random.choice(noise_file_list)
     ## [TODO]Adjust SNR 0-30 from google paper
    output_mixed_file, snr = make_noisy_speech(clean_speech_file, noise_file, 
                                               clean_data_dir, noise_dir, out_dir, 0, 30, None) 
    output_mixed_file_shortpath =  output_mixed_file.split(out_dir)[1]
    
#     with open('/home/fjia/data/google_dataset_v2/google_speech_recognition_v2_noisy/training_list.txt', 'a') as f:
#         f.write(output_mixed_file_shortpath) 
#         f.write('\n')
        
    return output_mixed_file_shortpath

NameError: name 'noise_file_list_path' is not defined

In [13]:
with open(noise_file_list_path) as f:
    noise_file_list = f.read().splitlines() 
noise_file = random.choice(noise_file_list)

NameError: name 'noise_file_list_path' is not defined

In [ ]:
output_mixed_file, snr = make_noisy_speech('/home/fjia/data/haorizi.wav', noise_file, 
                                           clean_data_dir, noise_dir, out_dir, 0, 30, None) 

In [8]:

y1, sample_rate = librosa.load('/home/fjia/data/haorizi.wav', sr=16000, mono=True)

# REPRODUCE
ipd.Audio(y1, rate = sample_rate)